In [1]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
import warnings 
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
df = pd.read_csv('../../data/processed/canomical_data.csv')
df.head()

,p_categories,p_brand,p_day_created,p_sold_quantity,p_original_price,p_discount_rate
0,7,155,659.0,702,528000,49
1,84,198,974.0,12844,799000,46
2,84,192,1372.0,938,209000,0
3,7,155,593.0,10359,473000,50
4,15,145,529.0,2338,106000,16


In [3]:
X = df.drop('p_sold_quantity', axis='columns')
y = testLabels = df.p_sold_quantity

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

In [4]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(StandardScaler(), PolynomialFeatures(degree), LinearRegression(**kwargs))

In [5]:
param_grid = {'polynomialfeatures__degree': [3,4,5], 'linearregression__fit_intercept': [True, False], 'linearregression__normalize': [True, False]}

In [6]:
gr_search = GridSearchCV(PolynomialRegression(), param_grid, 
                         cv=20, 
                         scoring='neg_mean_squared_error', 
                         verbose=3) 

In [7]:
gr_search.fit(X_train, y_train)

Fitting 20 folds for each of 12 candidates, totalling 240 fits
[CV 1/20] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-3334169.998 total time=   0.1s
[CV 2/20] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-1607712.444 total time=   0.3s
[CV 3/20] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-1849274.444 total time=   0.2s
[CV 4/20] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-8771551.465 total time=   0.1s
[CV 5/20] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-1853857.479 total time=   0.2s
[CV 6/20] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-2440217.632 total time=   0.1s
[CV 7/2

GridSearchCV(cv=20,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'linearregression__fit_intercept': [True, False],
                         'linearregression__normalize': [True, False],
                         'polynomialfeatures__degree': [3, 4, 5]},
             scoring='neg_mean_squared_error', verbose=3)

In [8]:
print('Best Score: %s' % gr_search.best_score_)
print('Best Hyperparameters: %s' % gr_search.best_params_)

Best Score: -1.2277947606474625e+19
Best Hyperparameters: {'linearregression__fit_intercept': False, 'linearregression__normalize': True, 'polynomialfeatures__degree': 3}


In [9]:
poly_grid = PolynomialRegression(degree=3, fit_intercept=False, normalize=True)

In [10]:
poly_grid.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('linearregression',
                 LinearRegression(fit_intercept=False, normalize=True))])

In [11]:
y_pred = poly_grid.predict(X_test)

In [12]:
from sklearn.metrics import mean_absolute_error
print('MSE: ', mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

MSE:  1013254.5587644045
RMSE:  1006.6054633094361
